# Molecular Dynamics with NequIP 

### Authors: Simon Batzner, Albert Musaelian, Lixin Sun, Anders Johansson, Boris Kozinsky

In [1]:
import warnings
import os

USE_COLAB = True
if USE_COLAB == True:
    from google.colab import drive
    drive.mount('/content/drive')
    work_dir = '/content/drive/MyDrive/Colab Notebooks/nequip/'
    data_dir = '/content/'
else:
    work_dir = '/Users/gabrieletocci/Google Drive/My Drive/Colab Notebooks/nequip/'
    data_dir = '/Users/gabrieletocci/Documents/projects/MD_DFT/'


Mounted at /content/drive


In [9]:
# Uncomment if running on google colab
! tar -xzvf  /content/drive/MyDrive/Colab\ Notebooks/MD_DFT/AIMD_data.tar.gz


._AIMD_data
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.google.drivefs.item-id#S'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
AIMD_data/
AIMD_data/._WATER-frc-10k-1.xyz
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.google.drivefs.item-id#S'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
AIMD_data/WATER-frc-10k-1.xyz
AIMD_data/._celldata.dat
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.google.drivefs.item-id#S'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
AIMD_data/celldata.dat
AIMD_data/._WATER-pos-10k-1.xyz
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.google.drivefs.item-id#S'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
AIMD_data/WATER-pos-10k-1.xyz


In [27]:
%%capture
import numpy as np
import pandas as pd
# install wandb
!pip install wandb
# install nequip
#uncomment if running on google colab
!git clone --depth 1 "https://github.com/gabriele16/nequip.git"
!pip install nequip/
# fix colab imports
import site
site.main()
# set to allow anonymous WandB
import os
os.environ["WANDB_ANONYMOUS"] = "must"
import numpy as np
import torch 
from ase.io import read, write
np.random.seed(0)
torch.manual_seed(0)

In [28]:
def MD_reader_xyz(f, data_dir, no_skip = 0):
  filename = os.path.join(data_dir, f)
  fo = open(filename, 'r')
  natoms_str = fo.read().rsplit(' i = ')[0]
  natoms = int(natoms_str.split('\n')[0])
  fo.close()  
  fo = open(filename, 'r')
  samples = fo.read().split(natoms_str)[1:]
  steps = []
  xyz = []
  temperatures = []
  energies = []
  for sample in samples[::no_skip]:
     entries = sample.split('\n')[:-1]
     energies.append(float(entries[0].split("=")[-1]))
     temp = np.array([list(map(float, lv.split()[1:])) for lv in entries[1:]])
     xyz.append(temp[:,:])
  return natoms_str, np.array(xyz), np.array(energies)     

In [29]:
def MD_writer_xyz(positions,forces,cell_vec_abc,energies,
                  data_dir,f):

  filename = os.path.join(data_dir, f)
  fo = open(filename, 'w')

  for it, frame in enumerate(positions):
    natoms = len(frame)
    fo.write("{:5d}\n".format(natoms))
    fo.write('Lattice="{:.5f} 0.0 0.0 0.0 {:.5f} 0.0 0.0 0.0 {:.5f}" \
    Properties="species:S:1:pos:R:3:forces:R:3" \
    energy={:.10f} pbc="T T T"\n'.format(cell_vec_abc[0],cell_vec_abc[1],cell_vec_abc[2],energies[it])    
    )
    if it%100 == 0.0:
      print(it)
    force_frame = forces[it]

    fo.write("".join("{:8s} {:.8f} {:16.8f} {:16.8f}\
     {:16.8f} {:16.8f} {:16.8f}\n".format(frame[iat].symbol,
                                          frame[iat].position[0],frame[iat].position[1],frame[iat].position[2],
                                          force_frame[iat].position[0],force_frame[iat].position[1],force_frame[iat].position[2])
                                          for iat in range(len(frame))))

In [30]:
def read_cell(f,data_dir):
  filename = os.path.join(data_dir,f)
  fo = open(filename,'r')
  cell_list_abc = fo.read().split('\n')[:-1]
  cell_vec_abc = np.array([list(map(float, lv.split())) for lv in cell_list_abc]).squeeze()
  return(cell_vec_abc)

cell_vec_abc = read_cell('celldata.dat',data_dir + '/AIMD_data')
cell_vec_abc

array([9.85, 9.85, 9.85])

In [31]:
wat_traj = read(data_dir +'AIMD_data/WATER-pos-10k-1.xyz',index=':')
wat_frc = read(data_dir + 'AIMD_data/WATER-frc-10k-1.xyz', index=':')

In [32]:
natoms, positions, energies = MD_reader_xyz('WATER-pos-10k-1.xyz', data_dir + '/AIMD_data/', no_skip=1)

In [33]:
MD_writer_xyz(wat_traj, wat_frc, cell_vec_abc, energies, data_dir + '/AIMD_data/', 'wat_pos_frc-10k.extxyz')


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


### Turn on GPU

Make sure Runtime --> Change runtime type is set to GPU

In [34]:
%%capture
# compile lammps
!wget "https://github.com/lammps/lammps/archive/stable.zip"
!unzip -q stable.zip
!rm stable.zip
!mv lammps-stable lammps
!wget "https://github.com/mir-group/pair_nequip/archive/main.zip"
!unzip -q main.zip
!rm main.zip
!mv pair_nequip-main pair_nequip
!cd pair_nequip && ./patch_lammps.sh ../lammps
!pip install mkl mkl-include
!cd lammps && mkdir -p build && cd build && cmake ../cmake -DCMAKE_PREFIX_PATH=`python -c 'import torch;print(torch.utils.cmake_prefix_path)'` && make -j4

## 3 Steps: 
* Train: using a data set, train the neural network 🧠 
* Deploy: convert the Python-based model into a stand-alone potential file for fast execution ⚡
* Run: run Molecular Dynamics, Monte Carlo, Structural Minimization, ...  with it in LAMMPS 🏃

<img src="https://github.com/mir-group/nequip_mrs_tutorial/blob/master/all.png?raw=true" width="60%">

### Train a model

<img src="https://github.com/mir-group/nequip_mrs_tutorial/blob/master/train.png?raw=true" width="60%">

This tutorial is set up to use `wandb` in anonymous mode; when you use NequIP yourself you will be presented with a login prompt.

Here, we will train a NequIP potential on the following system

* Toluene
* sampled at T=500K from AIMD
* at CCSD(T) accuracy (gold standard of quantum chemistry)
* Using 100 training configurations
* The units of the reference data are in kcal/mol and A. If you're more familiar with eV, remember 1 kcal/mol is chemical accuracy and is approximately 43 meV

Start a training run: this will print output to our console, but it is usually more convenient to view the results in a web interface called Weights and Biases. Click the link next to the rocket emoji to watch the run in the WandB interface 🚀 

In WandB, watch the followingkeys:

* Plot 1: validation_all_f_mae, training_all_f_mae
* Plot 2: validation_e/N_mae, training_e/N_mae

These are the validation/training error in all force components and the validation/training error in the potential energy, normalized by the number of atoms, respectively. 

In [39]:
from nequip.utils import Config
config = Config.from_file(work_dir + 'configs/my-full-example.yaml')
config

{'root': 'results/water', 'run_name': 'example-run-water', 'seed': 123, 'dataset_seed': 456, 'append': True, 'default_dtype': 'float32', 'allow_tf32': False, 'r_max': 4.0, 'num_layers': 4, 'l_max': 1, 'parity': True, 'num_features': 32, 'nonlinearity_type': 'gate', 'resnet': False, 'nonlinearity_scalars': {'e': 'silu', 'o': 'tanh'}, 'nonlinearity_gates': {'e': 'silu', 'o': 'tanh'}, 'num_basis': 8, 'BesselBasis_trainable': True, 'PolynomialCutoff_p': 6, 'invariant_layers': 2, 'invariant_neurons': 64, 'avg_num_neighbors': 'auto', 'use_sc': True, 'dataset': 'ase', 'dataset_file_name': './AIMD_data/wat_pos_frc-10k.extxyz', 'ase_args': {'format': 'extxyz'}, 'include_keys': ['user_label'], 'key_mapping': {'user_label': 'label0'}, 'verbose': 'info', 'log_batch_freq': 1, 'log_epoch_freq': 1, 'save_checkpoint_freq': -1, 'save_ema_checkpoint_freq': -1, 'n_train': 100, 'n_val': 50, 'learning_rate': 0.005, 'batch_size': 5, 'validation_batch_size': 10, 'max_epochs': 100000, 'train_val_split': 'rand

In [40]:
work_dir

'/content/drive/MyDrive/Colab Notebooks/nequip/'

In [41]:
! pwd

/content


In [ ]:
!ls /content/drive/MyDrive/Colab\ \Notebooks/nequip/configs/my-full-example.yaml
!rm -rf ./results
!nequip-train /content/drive/MyDrive/Colab\ \Notebooks/nequip/configs/my-full-example.yaml

'/content/drive/MyDrive/Colab Notebooks/nequip/configs/my-full-example.yaml'
Torch device: cuda
Processing dataset...
Loaded data: Batch(atomic_numbers=[960000, 1], batch=[960000], cell=[10000, 3, 3], edge_cell_shift=[25556838, 3], edge_index=[2, 25556838], forces=[960000, 3], pbc=[10000, 3], pos=[960000, 3], ptr=[10001], total_energy=[10000, 1])
Cached processed data to disk
Done!
Successfully loaded the data set of type ASEDataset(10000)...
Replace string dataset_forces_rms to 0.7736424803733826
Replace string dataset_per_atom_total_energy_mean to -5.736269474029541
Atomic outputs are scaled by: [H, O: 0.773642], shifted by [H, O: -5.736269].
Replace string dataset_forces_rms to 0.7736424803733826
Initially outputs are globally scaled by: 0.7736424803733826, total_energy are globally shifted by None.
Successfully built the network...
Number of weights: 154200
! Starting training ...

validation
# Epoch batch         loss       loss_f       loss_e        f_mae       f_rmse      H_f_ma

We see that the model has converged to an energy accuarcy < 1meV/atom and a force accuracy of approx. 40 meV/A within 5 minutes and trained on only 100 samples. That should give us a good first potential! Note that these numbers will decrease significantly if you increase the training set size and the number of epochs to train. 

### Deploy the model

<img src="https://github.com/mir-group/nequip_mrs_tutorial/blob/master/deploy.png?raw=true" width="60%">

We now convert the model to a potential file. This makes it independent of NequIP and we can use it any downstream application, such as LAMMPS. 

In [ ]:
!nequip-deploy build results/toluene/example-run-toluene toluene-deployed.pth

INFO:root:Atomic outputs are scaled by: 1.0, shifted by 0.0.
INFO:root:Compiled & optimized model.


## Evaluate Test Error on all remaining frames

Before running inference, we'd like to know how well the model is doing on a hold-out test set. We run the nequip-evaluate command to compute the test error on all data that we didn't use for training or validation. 

In [ ]:
!nequip-evaluate --train-dir results/toluene/example-run-toluene --batch-size 50

Using device: cuda
Loading model... 
loaded model from training session
Loading original dataset...
Loaded dataset specified in config.yaml.
Using origial training dataset (1000 frames) minus training (100 frames) and validation frames (50 frames), yielding a test set size of 850 frames.
Starting...
  0% 0/850 [00:00<?, ?it/s]

  6% 50/850 [00:00<00:04, 198.25it/s]
 12% 100/850 [00:00<00:07, 95.56it/s]
 18% 150/850 [00:02<00:14, 46.79it/s]
 24% 200/850 [00:04<00:17, 37.76it/s]

 35% 300/850 [00:04<00:07, 76.45it/s]

 47% 400/850 [00:04<00:03, 125.72it/s]


 65% 550/850 [00:04<00:01, 217.01it/s]


 82% 700/850 [00:04<00:00, 319.80it/s]


100% 850/850 [00:05<00:00, 167.55it/s]


--- Final result: ---
             0_f_mae =  0.796981           
             1_f_mae =  1.318640           
           all_f_mae =  1.057810           
             e/N_mae =  0.022260           
             0_f_mae =  0.796981           
             1_f_mae =  1.318640           
           all_f_mae =  1.05

Again, energy errors of < 1meV/atom (converted from kcal/mol to eV), and force errors of ~45 meV/A 🎉

# LAMMPS

We are now in a position to run MD with our potential. Here, we will minimize the geometry of the toluene molecule we trained on from a perturbed initial state. 

<img src="https://github.com/mir-group/nequip_mrs_tutorial/blob/master/run.png?raw=true" width="60%">

Set up a simple LAMMPS input file

CAUTION: the reference data here are in kcal/mol for the energies and kcal/mol/A for the forces. The NequIP model will therefore also be predicting outputs in these units. We are therefore using `units real` in LAMMPS (see [docs](https://docs.lammps.org/units.html)). If your reference data are in other units, you should using the corresponding units command in LAMMPS (e.g. if you use eV, A then `units metal` would be appropriate, which would then also change time units from `fs` to `ps`).

In [ ]:
lammps_input_minimize = """
units	real
atom_style atomic
newton off
thermo 1
read_data structure.data

pair_style	nequip
pair_coeff	* * ../toluene-deployed.pth C H 
mass            1 15.9994
mass            2 1.00794

neighbor 1.0 bin
neigh_modify delay 5 every 1

minimize 0.0 1.0e-8 10000 1000000
write_dump all custom output.dump id type x y z fx fy fz
"""
!mkdir lammps_run
with open("lammps_run/toluene_minimize.in", "w") as f:
    f.write(lammps_input_minimize)

Here's starting configuration for Toluene at CCSD(T) accuracy. We will strongly perturb the inital positions by sampling from a uniform distribution $\mathcal{U}([0, 0.5])$

In [ ]:
toluene_example = """15
 Lattice="100.0 0.0 0.0 0.0 100.0 0.0 0.0 0.0 100.0" Properties=species:S:1:pos:R:3 -169777.5840406276=T pbc="F F F"
 C       52.48936904      49.86911725      50.09520748
 C       51.01088202      49.89609925      50.17978049
 C       50.36647401      50.04650925      48.96054247
 C       48.95673398      50.29576626      48.71580846
 C       48.04533296      50.26023426      49.82589448
 C       48.70932398      49.85770925      51.01923950
 C       50.06326400      49.77782925      51.25691751
 H       52.94467905      50.48672926      50.86545150
 H       52.89060405      48.87175023      50.14480949
 H       53.02173405      50.05890725      49.03968247
 H       51.01439802      50.38234726      48.05314045
 H       48.80598498      50.64314926      47.68195744
 H       46.96754695      50.20586626      49.53998848
 H       48.16716997      49.75850325      51.88622952
 H       50.45791001      49.55387424      52.15303052
 """

with open('toluene.xyz', 'w') as f: 
    f.write(toluene_example)

# read as ASE objects
atoms = read('toluene.xyz', format='extxyz')

# perturb positions
p = atoms.get_positions()
p += np.random.rand(15, 3) * 0.5
atoms.set_positions(p)
atoms.set_pbc(False)

# write to a LAMMPS file
write("lammps_run/structure.data", atoms, format="lammps-data")

### Run the LAMMPS command: 

In [ ]:
!cd lammps_run/ && ../lammps/build/lmp -in toluene_minimize.in

LAMMPS (29 Sep 2021 - Update 2)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0.0000000 0.0000000 0.0000000) to (100.00000 100.00000 100.00000)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  15 atoms
  read_data CPU = 0.002 seconds
NEQUIP is using device cuda
NequIP Coeff: type 1 is element C
NequIP Coeff: type 2 is element H
Loading model from ../toluene-deployed.pth
Freezing TorchScript model...
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 5
  ghost atom cutoff = 5
  binsize = 2.5, bins = 40 40 40
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair nequip, perpetual
      attributes: full, newton off, ghost
      pair build: full/bin/ghost
      stencil: full/ghost/bin/3d
      bin: standard
Setting up cg style minimization ...
  Unit style   

We see LAMMPS converges quickly to a minimum. Let's check how well we did. 

In [ ]:
# read the final structure back in 
minimized = read('./lammps_run/output.dump', format='lammps-dump-text')

### Compare optimized bond length to true coupled cluster reference from CCCBDB

In [ ]:
# get distances of optimized geometry (reference data: CCSD(T) [Psi4, cc-pVDZ])
d_12 = minimized.get_distances(1, 2)

# reference: https://cccbdb.nist.gov/geom3x.asp?method=6&basis=2, coupled cluster
d_12_ccd = 1.4086

print('Relative Error in bond length w.r.t. Coupled Cluster from CCCBDB: {:.3f}%'.format((100 * np.abs(d_12 - d_12_ccd) / d_12_ccd)[0]))

Relative Error in bond length w.r.t. Coupled Cluster from CCCBDB: 0.100%


We find a final relative error close to Coupled Cluster accuracy 🎉

## Next Steps

This concludes our tutorial. A next step would be to head over to https://github.com/mir-group/nequip, install NequIP and get started with your own system. If you have questions, please don't hesitate to reach out to batzner@g.harvard.edu, we're happy to help! 

